In [140]:
import pandas as pd
import datetime 
import win32com.client
import requests
from bs4 import BeautifulSoup as bs

In [145]:
def get_raw_data(short_code, st_date, end_date):
    num_code = short_code[1:]
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'X-Requested-With': 'XMLHttpRequest'
    }
    url = 'https://www.investing.com/instruments/HistoricalDataAjax'
    payload = {
        'curr_id': '43433',
        'smlID': '25193974',
        'interval_sec': 'Daily',
        'sort_col': 'date',
        'sort_ord': 'DESC',
        'action': 'historical_data'
    }
    def tr_date(date):
        year ,month, day = date[:4], date[4:6], date[6:]
        tr_date = month + '/' + day + '/' + year
        return tr_date

    payload['header'] = num_code + ' ' + 'Historical Data'
    payload['st_date'] = tr_date(st_date)
    payload['end_date'] = tr_date(end_date)
    
    r1 = requests.post(url, data=payload, headers=header)
    tables = bs(r1.text,'lxml').findAll('table')
    raw_data = pd.read_html(str(tables[0]))[0]
    return raw_data

def preprocess(short_code, raw_data):
    # columns 정리
    raw_data.drop(['Open', 'High', 'Low', 'Vol.', 'Change %'], axis = 'columns', inplace = True)
    raw_data.rename(columns = {'Date' : 'date', 'Price' : 'adj'}, inplace = True)
    raw_data['short_code'] = short_code
    def tr_date_1(date_1):        
        tr_month = {
            'Jan' : 1,
            'Feb' : 2,
            'Mar' : 3,
            'Apr' : 4,
            'May' : 5,
            'Jun' : 6,
            'Jul' : 7,
            'Aug' : 8,
            'Sep' : 9,
            'Oct' : 10,
            'Nov' : 11,
            'Dec' : 12
        }
        year ,month, day = int(date_1[-4:]), tr_month[date_1[:3]], int(date_1[4:6])
        tr_date_var = datetime.date(year, month, day)
        return tr_date_var
    
    raw_data['date'] = raw_data['date'].apply(lambda x : tr_date_1(x))
    fined_data = raw_data
    return fined_data    

def get_adj_stock(short_code):
    raw_data = get_raw_data(short_code, '20180122', '20200120')
    fined_data = preprocess(short_code, raw_data)
    return fined_data



In [130]:
all_total = pd.read_csv('./all_total.csv')

In [136]:
all_total['short_code'].unique()

array(['A060310', 'A095570', 'A006840', ..., 'A003280', 'A037440',
       'A238490'], dtype=object)

In [ ]:
adj_list = []
for short_code_1 in all_total['short_code'].unique():
    try:
        print(short_code_1)
        adj_list.append(get_adj_stock(short_code_1))
    except Exception as e:
        print(e)
try:
    adj_total = pd.concat(adj_list)
    new_all_total = pd.merge(all_total, adj_total, on = ['date'])
except Exception as e:
        print(e)

A060310
A095570
A006840
A054620
A265520
A211270
A027410
A282330
A138930
A001460
A001040
A079160
A035760
A000120
A011150
A097950
A051500
A058820
A023460
A065770
A083660
A000590
A012030
A306620
A322780
A016610
A005830
A000990
A139130
A068790
A004840
A241520
A155660
A069730
A017940
A245620


In [128]:
fined_data.head(3)

,date,adj,short_code
0,2018-12-28,38700,A005930
1,2018-12-27,38250,A005930
2,2018-12-26,38350,A005930


In [96]:
pd.read_html(str(tables[0]))[0].iloc[0,0]

'Jan 23, 2020'

In [ ]:
## 함수의 형태
입력 : get_adj('A005930', '20180101', '20190101')
출력 : date, adj, short_code가 columns인 데이터 프레임을 반환한다.

In [125]:
def get_raw_data(short_code, st_date, end_date):
    num_code = short_code[1:]
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'X-Requested-With': 'XMLHttpRequest'
    }
    url = 'https://www.investing.com/instruments/HistoricalDataAjax'
    payload = {
        'curr_id': '43433',
        'smlID': '25193974',
        'interval_sec': 'Daily',
        'sort_col': 'date',
        'sort_ord': 'DESC',
        'action': 'historical_data'
    }
    def tr_date(date):
        year ,month, day = date[:4], date[4:6], date[6:]
        tr_date = month + '/' + day + '/' + year
        return tr_date

    payload['header'] = num_code + ' ' + 'Historical Data'
    payload['st_date'] = tr_date(st_date)
    payload['end_date'] = tr_date(end_date)
    
    r1 = requests.post(url, data=payload, headers=header)
    tables = bs(r1.text,'lxml').findAll('table')
    raw_data = pd.read_html(str(tables[0]))[0]
    return raw_data

In [142]:
def preprocess(short_code, raw_data):
    # columns 정리
    raw_data.drop(['Open', 'High', 'Low', 'Vol.', 'Change %'], axis = 'columns', inplace = True)
    raw_data.rename(columns = {'Date' : 'date', 'Price' : 'adj'}, inplace = True)
    raw_data['short_code'] = short_code
    def tr_date_1(date_1):        
        tr_month = {
            'Jan' : 1,
            'Feb' : 2,
            'Mar' : 3,
            'Apr' : 4,
            'May' : 5,
            'Jun' : 6,
            'Jul' : 7,
            'Aug' : 8,
            'Sep' : 9,
            'Oct' : 10,
            'Nov' : 11,
            'Dec' : 12
        }
        year ,month, day = int(date_1[-4:]), tr_month[date_1[:3]], int(date_1[4:6])
        tr_date_var = datetime.date(year, month, day)
        return tr_date_var
    
    raw_data['date'] = raw_data['date'].apply(lambda x : tr_date_1(x))
    fined_data = raw_data
    return fined_data    

In [127]:
raw_data = get_raw_data('A005930', '20180101', '20190101')
fined_data = preprocess('A005930', raw_data)
fined_data.head(3)

,date,adj,short_code
0,2018-12-28,38700,A005930
1,2018-12-27,38250,A005930
2,2018-12-26,38350,A005930


In [117]:
date(2020,1,23)

datetime.date(2020, 1, 23)

In [107]:
a = pd.read_html(str(tables[0]))[0]

### Merge 연습

In [19]:
import pandas as pd
import datetime

In [13]:
a1 = pd.DataFrame({'date' : [1,2,3,1,2,3], 'price' : [15,23,54,26,82,36], 'code': [1,1,1,2,2,2]})

In [14]:
a2 = pd.DataFrame({'date' : [1,2,3,1,2,3], 'vol' : [95,14,47,85,29,63], 'code': [1,1,1,2,2,2]})

In [16]:
pd.merge(a1,a2, on =['date', 'code'])

,date,price,code,vol
0,1,15,1,95
1,2,23,1,14
2,3,54,1,47
3,1,26,2,85
4,2,82,2,29
5,3,36,2,63


In [17]:
a2.to_pickle('test.pickle')
pd.read_pickle('test.pickle')

,date,vol,code
0,1,95,1
1,2,14,1
2,3,47,1
3,1,85,2
4,2,29,2
5,3,63,2


In [21]:
d = []
for i in range(1,7):
    d.append(datetime.datetime(2020,1,i))
a2['date'] = d
print(a2)
a2.to_pickle('test.pickle')
test = pd.read_pickle('test.pickle')
test.info()

        date  vol  code
0 2020-01-01   95     1
1 2020-01-02   14     1
2 2020-01-03   47     1
3 2020-01-04   85     2
4 2020-01-05   29     2
5 2020-01-06   63     2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
date    6 non-null datetime64[ns]
vol     6 non-null int64
code    6 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 208.0 bytes
